In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np 
from tensorflow import keras


In [10]:
train = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)
validation = ImageDataGenerator(rescale=1./255)
test = ImageDataGenerator(rescale=1./255)

In [11]:
train_dataset = train.flow_from_directory(
   'dataset-splitted/training-set/',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=32)
validation_dataset = validation.flow_from_directory(
     'dataset-splitted/validation-set/',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=32)
test_dataset = test.flow_from_directory(
    'dataset-splitted/test-set/',
    target_size=(224,224),
    batch_size=32, 
    shuffle=False)

Found 1884 images belonging to 6 classes.
Found 259 images belonging to 6 classes.
Found 384 images belonging to 6 classes.


In [12]:
train_dataset.class_indices

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}

In [13]:
model=keras.models.Sequential([keras.layers.Conv2D(16,(3,3),activation='relu', input_shape=(224,224,3)),
                                 keras.layers.MaxPool2D(2,2),
                                 keras.layers.Conv2D(32,(3,3),activation='relu'),
                                 keras.layers.MaxPool2D(2,2),
                                 keras.layers.Conv2D(64,(3,3),activation='relu'),
                                 keras.layers.MaxPool2D(2,2),
                                 keras.layers.Conv2D(128,(3,3),activation='relu'),
                                 keras.layers.MaxPool2D(2,2),
                                 keras.layers.Conv2D(256,(3,3),activation='relu'),
                                 keras.layers.MaxPool2D(2,2),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(4096,activation='relu'),
                                 keras.layers.Dense(6,activation='softmax') #softmax for multiclass
                                 ])

In [14]:
model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
             metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 128)      

In [16]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [17]:
model_fit=model.fit(train_dataset,
                   epochs=50,
                   validation_data=validation_dataset)

Epoch 1/50
59/59 [==============================] - 50s 853ms/step - loss: 1.7034 - accuracy: 0.2728 - val_loss: 1.3983 - val_accuracy: 0.4633
Epoch 2/50
59/59 [==============================] - 51s 866ms/step - loss: 1.4022 - accuracy: 0.4156 - val_loss: 1.2603 - val_accuracy: 0.4710
Epoch 3/50
59/59 [==============================] - 53s 890ms/step - loss: 1.2886 - accuracy: 0.4873 - val_loss: 1.2273 - val_accuracy: 0.5714
Epoch 4/50
59/59 [==============================] - 50s 848ms/step - loss: 1.1908 - accuracy: 0.5271 - val_loss: 1.0530 - val_accuracy: 0.5830
Epoch 5/50
59/59 [==============================] - 55s 933ms/step - loss: 1.0956 - accuracy: 0.5812 - val_loss: 1.1497 - val_accuracy: 0.5676
Epoch 6/50
59/59 [==============================] - 56s 951ms/step - loss: 1.0939 - accuracy: 0.5791 - val_loss: 1.0520 - val_accuracy: 0.5869
Epoch 7/50
59/59 [==============================] - 54s 919ms/step - loss: 0.9864 - accuracy: 0.6412 - val_loss: 0.9581 - val_accuracy: 0.6641

In [18]:
probabilities=model.predict(test_dataset)

In [19]:
model.load_weights


<bound method Model.load_weights of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001D742A39910>>

In [26]:
!mkdir -p saved_model
model.save('saved_model/model1')

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file saved_model already exists.
Error occurred while processing: saved_model.


INFO:tensorflow:Assets written to: saved_model/model1\assets


In [30]:
model.save('model.h5')
